<a href="https://colab.research.google.com/github/Kostogot/Two/blob/main/Binan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
 import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) 
B = pd.read_excel('/binan.xlsx', 1)
#B

In [10]:
 
P = [] 
for n in range(0,4):
  if B['operation'][n] == 'buy':
    B['%'][n] = -B['fee'][n] 
    B['ostatok'][n] = B['volume'][n] 
    P.append([B['price'][n],n])
  else:
    P.sort()
    V = B['volume'][n]
    B['%'][n] = 0
    for i in range(0,len(P)):
      if B['ostatok'][P[i][1]] > 0 and V > 0:
        if V > B['ostatok'][P[i][1]]:
          B['%'][n] += (B['price'][n]-B['price'][P[i][1]])*B['ostatok'][P[i][1]]
          V -= B['ostatok'][P[i][1]]
          B['ostatok'][P[i][1]] = 0
        else:
          B['%'][n] += (B['price'][n]-B['price'][P[i][1]])*V
          B['ostatok'][P[i][1]] -= V
          V = 0
    B['%'][n] -= B['fee'][n] 
#B

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykerne